<a href="https://colab.research.google.com/github/Ruchi-23/medicall/blob/main/medchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
!pip3 install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def tokenize(sentence):
    
    return nltk.word_tokenize(sentence)


def stem(word):
   
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
   
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [ ]:
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
with open('/content/drive/MyDrive/general_convo.json', 'r') as f:
    intents2 = json.load(f)

In [ ]:
all_words = []
tags = []
xy = []

In [ ]:
intents2

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hey :-)',
    'Hello, thanks for visiting',
    'Hi there, what can I do for you?',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'funny',
   'patterns': ['Tell me a joke!',
    'Tell me something funny!',
    'Do you know a joke?',
    "I don't feel happy",
    'Uplift my mood',
    'I feel sad',
    'Lift my spirits'],
   'responses': ["Here's a joke for you - Why did the hipster burn his mouth? He drank the coffee before it was cool.",
    "Here's a joke for you - What did the buffalo say

In [ ]:
for intent in intents2['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))
ignore_words = ['?', '.', '!',' ','+','-','%',';','`','""','/','°', '’',"'", '(', ')', ',',':', '``']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

100 patterns
20 tags: ['Asthma', 'Cardiac arrest', 'Dengue fever', 'Diabetes', 'Eating disorder', 'Fatty liver', 'Find doctors', 'Glaucoma', 'Joint pain', 'Liver cancer', 'Loss of apetite', 'Obesity', 'Panic disorder', 'Tooth ache', 'Vitamin deficiency', 'diagnostic centres', 'funny', 'goodbye', 'greeting', 'thanks']
172 unique stemmed words: ["'m", "'s", '10', 'a', 'abdomin', 'abus', 'ach', 'after', 'alcohol', 'alt', 'am', 'aminotransferas', 'an', 'and', 'anxieti', 'anxiou', 'anyon', 'apetit', 'appear', 'appetit', 'are', 'aspart', 'ast', 'bad', 'been', 'bmi', 'bodi', 'breath', 'breathless', 'build-up', 'bye', 'ca', 'calcul', 'call', 'cardiologist', 'caviti', 'centr', 'chang', 'check', 'chest', 'clinic', 'cloud', 'color', 'cough', 'count', 'day', 'decay', 'decreas', 'depress', 'diagnos', 'diagnost', 'difficulti', 'diminish', 'dizzi', 'do', 'doctor', 'due', 'eat', 'everi', 'excess', 'exhaust', 'eye', 'fat', 'fatti', 'feel', 'fever', 'find', 'for', 'forget', 'frequent', 'funni', 'get', '

In [ ]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [ ]:
num_epochs = 500
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

172 20


In [ ]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [ ]:
dataset = ChatDataset()

In [ ]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print(f'final loss: {loss.item():.4f}')


Epoch [1/500], Loss: 3.0116
Epoch [2/500], Loss: 3.0984
Epoch [3/500], Loss: 3.1422
Epoch [4/500], Loss: 3.0991
Epoch [5/500], Loss: 3.0134
Epoch [6/500], Loss: 3.0933
Epoch [7/500], Loss: 3.1426
Epoch [8/500], Loss: 2.9228
Epoch [9/500], Loss: 2.8739
Epoch [10/500], Loss: 3.0537
Epoch [11/500], Loss: 2.9659
Epoch [12/500], Loss: 2.9038
Epoch [13/500], Loss: 2.9828
Epoch [14/500], Loss: 3.1100
Epoch [15/500], Loss: 2.9222
Epoch [16/500], Loss: 2.9025
Epoch [17/500], Loss: 2.9595
Epoch [18/500], Loss: 2.7180
Epoch [19/500], Loss: 3.0609
Epoch [20/500], Loss: 2.7095
Epoch [21/500], Loss: 2.6809
Epoch [22/500], Loss: 2.4892
Epoch [23/500], Loss: 2.5854
Epoch [24/500], Loss: 2.7737
Epoch [25/500], Loss: 2.8511
Epoch [26/500], Loss: 2.9816
Epoch [27/500], Loss: 2.9919
Epoch [28/500], Loss: 3.0279
Epoch [29/500], Loss: 2.9418
Epoch [30/500], Loss: 2.7834
Epoch [31/500], Loss: 2.2192
Epoch [32/500], Loss: 2.5383
Epoch [33/500], Loss: 2.5750
Epoch [34/500], Loss: 2.6474
Epoch [35/500], Loss: 2

In [ ]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


In [ ]:
FILE = "data.pth"
data = torch.load(FILE)

In [ ]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]


In [ ]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=172, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=20, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "MediCall"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        """for i in range(100):
            if tag == intents[i]["tag"]:
                print(f"{bot_name}: {random.choice(intent[i]['Doctor'])}")"""
        for intent in intents2['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Sorry! I do not understand...")

Let's chat! (type 'quit' to exit)
You: Hello
MediCall: Hey :-)
You: I feel very anxious
MediCall: Sorry! I do not understand...
You: I'm feeling anxious
MediCall: You seem to be having a panic attack. Do not fight it. Stay where you are, if possible. Breathe slowly and deeply. Focus on positive, peaceful and relaxing images.
You: I'm sweating
MediCall: You seem to be having a panic attack. Do not fight it. Stay where you are, if possible. Breathe slowly and deeply. Focus on positive, peaceful and relaxing images.
You: I feel a sharp chest pain
MediCall: Chew and swallow an aspirin, unless you are allergic to aspirin or have been told by your doctor never to take aspirin. Cardiologists nearby : https://bit.ly/cardiologists_nearby , A hospital near you : http://pulsecardiologygroup.co.uk/sussex-cardiovascular-clinic/
You: What to do for asharp chest pain?
MediCall: Sorry! I do not understand...
You: What to do for a sharp chest pain?
MediCall: Sorry! I do not understand...
You: How to tr

In [ ]:
!pip install colabcode
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 KB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 K

In [ ]:
from fastapi import FastAPI

In [ ]:
!pip install fastapi uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {"Hello":"World"}

In [20]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://bb9a-34-67-87-73.ngrok.io


INFO:     Started server process [643]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     109.123.102.140:0 - "GET / HTTP/1.1" 200 OK
INFO:     109.123.102.140:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [643]


In [13]:
from pydantic import BaseModel

class request_body(BaseModel):
  sentence : str

  

In [ ]:
"""@app.post('/predict')
def predict(data : request_body):

    #class_idx = clf.predict(test_data)[0]
    #return { 'class' : iris.target_names[class_idx]}
    bot_name = "MediCall"
    print("Let's chat! (type 'quit' to exit)")
    while True:
    # sentence = "do you use credit cards?"
        sentence = [[data.sentence]]
        if sentence[0][0] == "quit":
            break

        sentence = tokenize(sentence[0][0])
        X = bag_of_words(sentence, all_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X).to(device)

        output = model(X)
        _, predicted = torch.max(output, dim=1)

        tag = tags[predicted.item()]

        probs = torch.softmax(output, dim=1)
        prob = probs[0][predicted.item()]
        if prob.item() > 0.75:
            for i in range(100):
                if tag == intents[i]["tag"]:
                    print(f"{bot_name}: {random.choice(intent[i]['Doctor'])}")
            for intent in intents2['intents']:
                if tag == intent["tag"]:
                    return {"MediCall" : random.choice(intent['responses'])}
        else:
            return {"MediCall" : "Sorry I don't understand."}
            """